In [411]:
import csv
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
from matplotlib import pyplot as plt

In [413]:
merged_df = pd.read_csv('chips_merged.csv')
print(merged_df.sort_values('Transaction_ID'))

              Date  Store_Number  Loyalty_Card_Number  Transaction_ID  \
0       2018-10-17             1                 1000               1   
240664  2018-09-16             1                 1002               2   
188931  2019-03-07             1                 1003               3   
188932  2019-03-08             1                 1003               4   
102787  2018-11-02             1                 1004               5   
...            ...           ...                  ...             ...   
25107   2018-09-26           272               272392          270206   
25108   2018-09-29           272               272392          270207   
25109   2018-10-31           272               272392          270208   
25110   2019-02-17           272               272392          270209   
15829   2018-12-20            88               237324         2415841   

        Product_Number                           Product_Name  \
0                    5        Natural Chip Compny SeaSalt1

In [415]:
control_df = pd.read_csv('chips_merged.csv')

control_df['Date'] = pd.to_datetime(control_df['Date'])

control_df['Numbered_Life_Stage'] = control_df['Life_Stage'].astype('category').cat.codes
control_df['Numbered_Customer_Type'] = control_df['Customer_Type'].astype('category').cat.codes

control_start, control_end = "2018-07-01", "2019-01-31"
control_dataframe = control_df[(control_df['Date'] >= control_start) & (control_df['Date'] <= control_end)]

num_months = 7

control_aggregated = control_dataframe.groupby('Store_Number').agg(
    Total_Sales=('Total_Sales', 'sum'),
    Total_Sales_Average=('Total_Sales', 'sum'),
    Number_of_Transactions=('Transaction_ID', 'count'),
    Average_Number_of_Transactions=('Transaction_ID', 'count'),
    Number_of_Unique_Customers=('Loyalty_Card_Number', 'nunique'),
    Average_Number_of_Unique_Customers=('Loyalty_Card_Number', 'nunique'),
    Average_Items_per_Transaction=('Product_Quantity', 'mean'),
    Average_Life_Stage=('Numbered_Life_Stage', 'mean'),
    Average_Customer_Type=('Numbered_Customer_Type', 'mean')
).reset_index()

control_aggregated['Total_Sales_Average'] /= num_months
control_aggregated['Average_Number_of_Transactions'] /= num_months
control_aggregated['Average_Number_of_Unique_Customers'] /= num_months

control_aggregated = control_aggregated[~control_aggregated['Store_Number'].isin([31, 11])]

control_aggregated.sort_values(by='Store_Number', inplace=True)

control_aggregated.dropna(inplace=True)

               Life_Stage  Numbered_Life_Stage
0  Midage singles/couples                  0.0
1            New families                  1.0
2          Older families                  2.0
3   Older singles/couples                  3.0
4                Retirees                  4.0
5          Young families                  5.0
6   Young singles/couples                  6.0
             Date  Store_Number  Loyalty_Card_Number  Transaction_ID  \
0      2018-10-17             1                 1000               1   
1      2019-05-14             1                 1307             348   
2      2018-11-10             1                 1307             346   
3      2019-03-09             1                 1307             347   
4      2019-05-20             1                 1343             383   
...           ...           ...                  ...             ...   
264830 2019-03-09           272               272319          270088   
264831 2018-08-13           272               27

In [340]:
trial_stores = [77, 86, 88]
non_trial_stores = control_aggregated[~control_aggregated['Store_Number'].isin(trial_stores)]['Store_Number'].unique()

metrics = ['Total_Sales', 'Total_Sales_Average', 'Number_of_Transactions', 'Average_Number_of_Transactions', 'Number_of_Unique_Customers',
            'Average_Number_of_Unique_Customers', 
           'Average_Items_per_Transaction', 
           'Average_Life_Stage', 'Average_Customer_Type']

In [341]:
from scipy.spatial.distance import euclidean

def find_best_control_distance(trial_store):
    best_store = None
    best_distance = float('inf')
    
    trial_data = control_aggregated[control_aggregated['Store_Number'] == trial_store].iloc[0, 1:]
    for store in non_trial_stores:
        control_data = control_aggregated[control_aggregated['Store_Number'] == store].iloc[0, 1:]
        
        if not control_data.empty:
            distance = euclidean(trial_data, control_data)
            if distance < best_distance:
                best_distance = distance
                best_store = store
    
    return best_store, best_distance

control_stores_distance = {trial_store: find_best_control_distance(trial_store) for trial_store in trial_stores}

for trial, (control, distance) in control_stores_distance.items():
    print(f"Trial Store {trial} -> Best Control Store: {control} (Distance: {distance:.4f})")

Trial Store 77 -> Best Control Store: 188 (Distance: 35.2295)
Trial Store 86 -> Best Control Store: 13 (Distance: 16.2734)
Trial Store 88 -> Best Control Store: 237 (Distance: 21.7403)


In [342]:
# Creating the dataframe containing only data for the trial period.

trial_df = pd.read_csv('chips_merged.csv')

trial_df['Date'] = pd.to_datetime(trial_df['Date'])

trial_df['Numbered_Life_Stage'] = trial_df['Life_Stage'].astype('category').cat.codes
trial_df['Numbered_Customer_Type'] = trial_df['Customer_Type'].astype('category').cat.codes

trial_start, trial_end = "2019-02-01", "2019-04-30"
trial_dataframe = trial_df[(trial_df['Date'] >= trial_start) & (trial_df['Date'] <= trial_end)]

num_months = 3

trial_aggregated = trial_dataframe.groupby('Store_Number').agg(
    Total_Sales=('Total_Sales', 'sum'),
    Total_Sales_Average=('Total_Sales', 'sum'), 
    Number_of_Transactions=('Transaction_ID', 'count'),
    Average_Number_of_Transactions=('Transaction_ID', 'count'),
    Number_of_Unique_Customers=('Loyalty_Card_Number', 'nunique'),
    Average_Number_of_Unique_Customers=('Loyalty_Card_Number', 'nunique'),
    Average_Items_per_Transaction=('Product_Quantity', 'mean'),
    Average_Life_Stage=('Numbered_Life_Stage', 'mean'),
    Average_Customer_Type=('Numbered_Customer_Type', 'mean')
).reset_index()

trial_aggregated['Total_Sales_Average'] /= num_months
trial_aggregated['Average_Number_of_Transactions'] /= num_months
trial_aggregated['Average_Number_of_Unique_Customers'] /= num_months

trial_aggregated = trial_aggregated[~trial_aggregated['Store_Number'].isin([31, 11])]

trial_aggregated.sort_values(by='Store_Number', inplace=True)

trial_aggregated.dropna(inplace=True)

In [344]:
# I created this function to compare the performance of metrics between a trial store and a control store during the trial period

def trial_comparison(trial_store, control_store):
    x = trial_aggregated[(trial_aggregated['Store_Number'] == trial_store) | (trial_aggregated['Store_Number'] == control_store)]
    results = {} 
    for metric in metrics:
            results[metric] = x.groupby('Store_Number')[metric].mean()
    return pd.DataFrame(results)

print(trial_comparison(77,81))

              Total_Sales  Total_Sales_Average  Number_of_Transactions  \
Store_Number                                                             
77                  777.0                259.0                   148.0   
81                 3597.9               1199.3                   406.0   

              Average_Number_of_Transactions  Number_of_Unique_Customers  \
Store_Number                                                               
77                                 49.333333                       124.0   
81                                135.333333                       246.0   

              Average_Number_of_Unique_Customers  \
Store_Number                                       
77                                     41.333333   
81                                     82.000000   

              Average_Items_per_Transaction  Average_Life_Stage  \
Store_Number                                                      
77                                 1.581081            

In [345]:
# I created this function to compare the performance of metrics between a trial store and a control store during the control period

def control_comparison(trial_store, control_store):
    x = control_aggregated[(control_aggregated['Store_Number'] == trial_store) | (control_aggregated['Store_Number'] == control_store)]
    results = {}  
    for metric in metrics:
            results[metric] = x.groupby('Store_Number')[metric].mean()
    return pd.DataFrame(results)
print(control_comparison(77,81))

              Total_Sales  Total_Sales_Average  Number_of_Transactions  \
Store_Number                                                             
77                 1699.0           242.714286                   317.0   
81                 8260.3          1180.042857                   954.0   

              Average_Number_of_Transactions  Number_of_Unique_Customers  \
Store_Number                                                               
77                                 45.285714                       239.0   
81                                136.285714                       356.0   

              Average_Number_of_Unique_Customers  \
Store_Number                                       
77                                     34.142857   
81                                     50.857143   

              Average_Items_per_Transaction  Average_Life_Stage  \
Store_Number                                                      
77                                 1.526814            

In [346]:
# Finally, this function takes the results of the previous two functions and calculates a percentage. 
# a score of 100 means the pre-trial performance is identical to the trial performance.
# over 100 means an increase, less than 100 means a decrease.

def trial_change(trial_store, control_store):
    x = (trial_comparison(trial_store, control_store) / control_comparison(trial_store, control_store))*100
    results = {}
    for metric in metrics:
        results[metric] = x.groupby('Store_Number')[metric].mean()
    return pd.DataFrame(results)

trial_results_77_188 = trial_change(77, 188)
print(trial_results_77_188)

trial_results_77_188.to_csv('77_to_188.csv'

              Total_Sales  Total_Sales_Average  Number_of_Transactions  \
Store_Number                                                             
77              45.732784           106.709829               46.687697   
188             53.757054           125.433125               50.511945   

              Average_Number_of_Transactions  Number_of_Unique_Customers  \
Store_Number                                                               
77                                108.937960                   51.882845   
188                               117.861206                   57.990868   

              Average_Number_of_Unique_Customers  \
Store_Number                                       
77                                    121.059972   
188                                   135.312024   

              Average_Items_per_Transaction  Average_Life_Stage  \
Store_Number                                                      
77                               103.554277           9

In [347]:
trial_results_86_13 = trial_change(86, 13)
print(trial_results_86_13)

              Total_Sales  Total_Sales_Average  Number_of_Transactions  \
Store_Number                                                             
13              47.884606           111.730747               46.927374   
86              45.559940           106.306527               46.258503   

              Average_Number_of_Transactions  Number_of_Unique_Customers  \
Store_Number                                                               
13                                109.497207                   77.642276   
86                                107.936508                   84.645669   

              Average_Number_of_Unique_Customers  \
Store_Number                                       
13                                    181.165312   
86                                    197.506562   

              Average_Items_per_Transaction  Average_Life_Stage  \
Store_Number                                                      
13                               100.336323           9

In [348]:
trial_results_88_237 = trial_change(88, 237)
print(trial_results_88_237)

              Total_Sales  Total_Sales_Average  Number_of_Transactions  \
Store_Number                                                             
88              45.683959           106.595905               44.916821   
237             40.747145            95.076671               40.074557   

              Average_Number_of_Transactions  Number_of_Unique_Customers  \
Store_Number                                                               
88                                104.805915                   69.786096   
237                                93.507300                   72.576177   

              Average_Number_of_Unique_Customers  \
Store_Number                                       
88                                    162.834225   
237                                   169.344414   

              Average_Items_per_Transaction  Average_Life_Stage  \
Store_Number                                                      
88                               100.791803           9

In [351]:
control_aggregated.to_csv("control_aggregated.csv", index=False)
trial_aggregated.to_csv("trial_aggregated.csv", index=False)
trial_results_77_188.to_csv("trial_results_77_188.csv", index=True)
trial_results_86_13.to_csv("trial_results_86_13.csv", index=True)
trial_results_88_237.to_csv("trial_results_88_237.csv", index=True)


In [417]:
merged_df['Date'] = pd.to_datetime(merged_df['Date'])

merged_df['Numbered_Life_Stage'] = merged_df['Life_Stage'].astype('category').cat.codes
merged_df['Numbered_Customer_Type'] = merged_df['Customer_Type'].astype('category').cat.codes

merged_df['Month'] = merged_df['Date'].dt.to_period('M')

full_year_aggregated = merged_df.groupby(['Store_Number', 'Month']).agg(
    Total_Sales=('Total_Sales', 'sum'),
    Total_Sales_Average=('Total_Sales', 'sum'),
    Number_of_Transactions=('Transaction_ID', 'count'),
    Average_Number_of_Transactions=('Transaction_ID', 'count'),
    Number_of_Unique_Customers=('Loyalty_Card_Number', 'nunique'),
    Average_Number_of_Unique_Customers=('Loyalty_Card_Number', 'nunique'),
    Average_Items_per_Transaction=('Product_Quantity', 'mean'),
    Average_Life_Stage=('Numbered_Life_Stage', 'mean'),
    Average_Customer_Type=('Numbered_Customer_Type', 'mean')
).reset_index()

full_year_aggregated.sort_values(by=['Store_Number', 'Month'], inplace=True)

full_year_aggregated.to_csv("full_year_aggregated.csv", index=False)